In [1]:
import requests

In [3]:
import json
import yfinance as yf
import psycopg2
import os
import boto3

# Set up environment variables for RDS connection
RDS_HOST = "ae31db6120e004afc9dd67a33d31fa9c-460277633.us-west-2.elb.amazonaws.com"
RDS_DB = "db4"
RDS_USER = "postgres"
RDS_PASSWORD = "postgres"
CLOUDWATCH_NAMESPACE = 'StockPriceMonitoring'

# Create a CloudWatch client
cloudwatch = boto3.client('cloudwatch')

def get_stock_price(symbol):
    """
    Fetches the latest stock price for the given symbol using yfinance.
    """
    stock = yf.Ticker(symbol)
    stock_info = stock.history(period='1d')
    return stock_info['Close'].iloc[-1]

def store_stock_price(symbol, price):
    """
    Stores the stock price in the PostgreSQL database.
    """
    try:
        price = float(price)
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(
            host=RDS_HOST,
            database=RDS_DB,
            user=RDS_USER,
            password=RDS_PASSWORD
        )
        cur = conn.cursor()
        
        # Insert stock price into the table
        cur.execute("INSERT INTO stock_prices (stock_symbol, stock_price) VALUES (%s, %s)", (symbol, price))
        conn.commit()
        cur.close()
        conn.close()
        print(f"Stored {symbol} stock price: {price} in the database.")
    except Exception as e:
        print(f"Error storing stock price: {e}")

def publish_cloudwatch_metric(symbol, price):
    """
    Publishes the stock price as a custom CloudWatch metric.
    """
    try:
        cloudwatch.put_metric_data(
            Namespace=CLOUDWATCH_NAMESPACE,
            MetricData=[
                {
                    'MetricName': 'StockPrice',
                    'Dimensions': [{'Name': 'StockSymbol', 'Value': symbol}],
                    'Value': price,
                    'Unit': 'None'
                }
            ]
        )
        print(f"Published {symbol} stock price: {price} to CloudWatch.")
    except Exception as e:
        print(f"Error publishing to CloudWatch: {e}")

def main():
    """
    Main function to get cryptocurrency and currency prices, store them in PostgreSQL,
    and send them to CloudWatch.
    """
    # List of cryptocurrency and currency symbols
    symbols = {
        "BTC-GBP": "Bitcoin (BTC)",
        "ETH-GBP": "Ethereum (ETH)",
        "EUR-GBP": "Euro (EUR)",
        "DOGE-GBP": "Dogecoin (DOGE)"
    }

    for ticker, name in symbols.items():
        try:
            # Fetch the price for each symbol
            price = get_stock_price(ticker)
            print(f"Fetched {name} price: {price}")
            
            # Store the price in PostgreSQL
            store_stock_price(ticker, price)
            
            # Publish the price to CloudWatch
            publish_cloudwatch_metric(ticker, price)
        except Exception as e:
            print(f"Error processing {name}: {e}")


if __name__ == "__main__":
    main()


Fetched Bitcoin (BTC) price: 48985.83203125
Stored BTC-GBP stock price: 48985.83203125 in the database.
Published BTC-GBP stock price: 48985.83203125 to CloudWatch.
Fetched Ethereum (ETH) price: 1916.54345703125
Stored ETH-GBP stock price: 1916.54345703125 in the database.
Published ETH-GBP stock price: 1916.54345703125 to CloudWatch.


$EUR-GBP: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Error processing Euro (EUR): single positional indexer is out-of-bounds
Fetched Dogecoin (DOGE) price: 0.08703538775444031
Stored DOGE-GBP stock price: 0.08703538775444031 in the database.
Published DOGE-GBP stock price: 0.08703538775444031 to CloudWatch.
